In [17]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 10.5 MB/s eta 0:00:00


In [ ]:
from ipywidgets import widgets
import pandas as pd
import io
import numpy as np
from keras.utils import to_categorical
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

In [ ]:
# Global variables
uploaded_content = None
model = load_model('/content/drive/MyDrive/Colab Notebooks/Model/cnn1d.keras')

# Fungsi yang akan dipanggil ketika tombol Analisis di click
def upload_button_click_handler(upload_n_analyze_button):
    global uploaded_content, model
    uploaded_file = next(iter(uploaded.value))
    uploaded_content = uploaded.value[uploaded_file]["content"]
    print(f"Berhasil upload file! Nama file: {uploaded_file}")

    if uploaded_content is None:
        print("Tolong upload file dahulu.")
        return

    # Konversi konten yang di upload menjadi pandas DataFrame / Load sampel data
    data_test_attack = pd.read_csv(io.BytesIO(uploaded_content))

    # Pre-processing data
    test_attack_cat = data_test_attack[['attack_cat']].copy()
    data_test_attack.drop('attack_cat', axis=1, inplace=True)
    X_test = data_test_attack.drop(columns=['dinpkt', 'ct_ftp_cmd', 'ct_src_dport_ltm', 'is_sm_ips_ports'])
    y_test = test_attack_cat

    # Encode kolom 'attack_cat'
    attack_cat_mapping = {
        "Analysis": 0,
        "Backdoor": 1,
        "DoS": 2,
        "Exploits": 3,
        "Fuzzers": 4,
        "Generic": 5,
        "Reconnaissance": 6,
        "Shellcode": 7,
        "Worms": 8
        }
    y_test = test_attack_cat['attack_cat'].map(attack_cat_mapping)
    num_class = 9
    y_test = to_categorical(y_test, num_class)
    X_test = np.array(X_test, dtype=np.float32)

    # Memprediksi menggunakan model
    predictions = model.predict(X_test)

    class_labels = ["Analysis", "Backdoor", "DoS", "Exploits", "Fuzzers", "Generic", "Reconnaissance", "Shellcode", "Worms"]

    predicted_class_index = np.argmax(predictions)
    predicted_class_label = class_labels[predicted_class_index]

    # Menampilkan hasil prediksi sampel berdasarkan hasil prediksi
    print(f"Model memprediksi input termasuk kelas {predicted_class_label}.")

    # Mengubah skor menjadi persentase
    prediction_scores = predictions.flatten()
    prediction_scores_percentage = prediction_scores * 100

    # Tampilkan semua label kelas dengan skor prediksi yang sesuai sebagai persentase
    print("Skor prediksi per kelas:")
    for i, score_percentage in enumerate(prediction_scores_percentage):
        print(f"{class_labels[i]}: {score_percentage:.2f}%")

    # Tampilkan batas
    print('=================== Selesai Menganalisis ===================')

# Create an upload button widget
uploaded = widgets.FileUpload(accept='*', multiple=False, description="Upload Sampel")

# Create an upload and analyze button
upload_n_analyze_button = widgets.Button(description="Analisis")
upload_n_analyze_button.on_click(upload_button_click_handler)

# Display the UI
widgets.VBox([uploaded, upload_n_analyze_button])